In [1]:
file_path = 'input/day16.txt'
with open(file_path, 'r') as file:
    real_data = file.read()


In [377]:
test_data="""###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############"""
print(test_data)
data = real_data

data = data.replace("\n","")
from math import sqrt
n = int(sqrt(len(data)))

def map_up(index, n):
    row = index // n
    col = index % n
    return int(row), int(col)

def map_down(x, y, n):
    return int(x * n + y)

start_pos = *map_up(data.find("S"),n),">"

target = map_up(data.find("E"),n)#,">"

g = dict()
for i,char  in enumerate(data):

    if char!="#":
        x,y = map_up(i, n)
        #rotate
        g[(x,y,"^")] = dict()
        g[(x,y,"^")][(x,y,"<")] = 1000
        g[(x,y,"^")][(x,y,">")] = 1000

        g[(x,y,">")] = dict()
        g[(x,y,">")][(x,y,"^")] = 1000
        g[(x,y,">")][(x,y,"v")] = 1000

        g[(x,y,"v")] = dict()
        g[(x,y,"v")][(x,y,">")] = 1000
        g[(x,y,"v")][(x,y,"<")] = 1000

        g[(x,y,"<")] = dict()
        g[(x,y,"<")][(x,y,"^")] = 1000
        g[(x,y,"<")][(x,y,"v")] = 1000


        #up
        if  x-1>=0 and data[map_down(x-1, y, n)]!="#":
            g[(x,y,"^")][(x-1,y,"^")] = 1

        #left    
        if 0<=y-1 and data[map_down(x, y-1, n)]!="#":
            g[(x,y,"<")][(x,y-1,"<")] = 1

        #right    
        if y+1<n and data[map_down(x, y+1, n)]!="#":
            g[(x,y,">")][(x,y+1,">")] = 1
            
        #down
        if x+1<n and data[map_down(x+1, y, n)]!="#":
            g[(x,y,"v")][(x+1,y,"v")] = 1



print(start_pos)

print(target)

###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
(139, 1, '>')
(1, 139)


In [378]:
import heapq

def dijkstra(g, source, target=None):

    # Priority queue for nodes to visit
    pq = []
    heapq.heappush(pq, (0, source))  # (distance, node)
    
    # Distance from source to each node
    distances = {node: float('inf') for node in g}
    distances[source] = 0

    # Previous node in the shortest path
    previous = {node: None for node in g}

    while pq:
        # Get the node with the smallest distance
        current_distance, current_node = heapq.heappop(pq)
        if current_distance > distances[current_node]:
            continue

        # Explore neighbors
        for neighbor, weight in g[current_node].items():
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous[neighbor] = current_node
                heapq.heappush(pq, (distance, neighbor))

    return distances, previous

dist, prev = dijkstra(g, start_pos)

In [216]:
def insert_newline_every_n_chars(input_string, n):
    return '\n'.join(input_string[i:i+n] for i in range(0, len(input_string), n))

def update_map(input_string, tup, n):
    if len(tup)==3:
        x,y,new_char = tup
    else:
        x,y = tup
        new_char = "O"
    index = map_down(x,y,n)
    return input_string[:index] + new_char + input_string[index + 1:]


map_vis = data
last_step = prev[(1,13,"^")]
target_found=False
while not target_found:
    map_vis = update_map(map_vis,last_step,n)
    last_step = prev[last_step]
    if last_step==start_pos:
        target_found=True
map_vis = insert_newline_every_n_chars(map_vis, n)
print(map_vis)


###############
#.......#....E#
#.#.###.#.###^#
#.....#.#...#^#
#.###.#####.#^#
#.#.#.......#^#
#.#.#####.###^#
#....^>>>>>>#^#
###.#^#####v#^#
#...#^....#v#^#
#.#.#^###.#v#^#
#^>>>>#...#v#^#
#^###.#.#.#v#^#
#^..#.....#v>>#
###############


In [ ]:
from tqdm import tqdm

target_dict = {(*target,c) : dist[(*target,c)] for c in "^>v<"}
tardet_node_with_dir = min(target_dict, key=target_dict.get)
min_cost = target_dict[tardet_node_with_dir]

nodes_to_search  = dict()


def manhattan_distance(point1, point2):
    return abs(point1[0] - point2[0]) + abs(point1[1] - point2[1])


def get_path_to_target(src, tardet_node_with_dir, prev):
    route = set()
    target_found = False
    last_step = tardet_node_with_dir
    while not target_found:
        if last_step==src:
            target_found=True
        route.add((last_step[:2]))
        last_step = prev[last_step]

    return route

#the first path
least_one = get_path_to_target(start_pos, tardet_node_with_dir, prev)

for node, cost in dist.items():
    if cost+manhattan_distance(node[:2],target)<min_cost:
        if  True:
            nodes_to_search[node]=cost
nodes_to_search = { key: value for key, value in nodes_to_search.items() if (key[0], key[1]) not in least_one }


#run dijkstra for with source=node_to_search
#run dijkstra for with source=node_to_search
#for node, cost_to_node  in  nodes_to_search.items():
for node, cost_to_node in tqdm(nodes_to_search.items(), desc="Processing nodes"):

    dist_tmp, prev_tmp = dijkstra(g, node)

    target_dict_tmp = {(*target,c) : dist_tmp[(*target,c)] for c in "^>v<"}
    tardet_node_with_dir_tmp = min(target_dict_tmp, key=target_dict_tmp.get)
    min_cost_tmp = target_dict[tardet_node_with_dir]

    tot=dist_tmp[tardet_node_with_dir_tmp ] + dist[node]
    if tot== min_cost:
        for step in get_path_to_target(node, tardet_node_with_dir_tmp, prev_tmp):
            #print(f"found one{node}")
            least_one.add(step)


print(len(least_one))

Processing nodes: 100%|██████████| 25739/25739 [22:47<00:00, 18.82it/s]

433
